# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources in the README.md file
- Happy learning!

In [3]:
# Import your libraries:

%matplotlib inline

import numpy as np
import pandas as pd

In this lab, we will explore a dataset that describes websites with different features and labels them either benign or malicious . We will use supervised learning algorithms to figure out what feature patterns malicious websites are likely to have and use our model to predict malicious websites.

# Challenge 1 - Explore The Dataset

Let's start by exploring the dataset. First load the data file:

In [4]:
websites = pd.read_csv('../data/website.csv')

#### Explore the data from an bird's-eye view.

You should already been very familiar with the procedures now so we won't provide the instructions step by step. Reflect on what you did in the previous labs and explore the dataset.

Things you'll be looking for:

* What the dataset looks like?
* What are the data types?
* Which columns contain the features of the websites?
* Which column contains the feature we will predict? What is the code standing for benign vs malicious websites?
* Do we need to transform any of the columns from categorical to ordinal values? If so what are these columns?

Feel free to add additional cells for your explorations. Make sure to comment what you find out.

In [5]:
# Your code here
websites.head()

,URL,URL_LENGTH,NUMBER_SPECIAL_CHARACTERS,CHARSET,SERVER,CONTENT_LENGTH,WHOIS_COUNTRY,WHOIS_STATEPRO,WHOIS_REGDATE,WHOIS_UPDATED_DATE,...,DIST_REMOTE_TCP_PORT,REMOTE_IPS,APP_BYTES,SOURCE_APP_PACKETS,REMOTE_APP_PACKETS,SOURCE_APP_BYTES,REMOTE_APP_BYTES,APP_PACKETS,DNS_QUERY_TIMES,Type
0,M0_109,16,7,iso-8859-1,nginx,263.0,None,None,10/10/2015 18:21,None,...,0,2,700,9,10,1153,832,9,2.0,1
1,B0_2314,16,6,UTF-8,Apache/2.4.10,15087.0,None,None,None,None,...,7,4,1230,17,19,1265,1230,17,0.0,0
2,B0_911,16,6,us-ascii,Microsoft-HTTPAPI/2.0,324.0,None,None,None,None,...,0,0,0,0,0,0,0,0,0.0,0
3,B0_113,17,6,ISO-8859-1,nginx,162.0,US,AK,7/10/1997 4:00,12/09/2013 0:45,...,22,3,3812,39,37,18784,4380,39,8.0,0
4,B0_403,17,6,UTF-8,None,124140.0,US,TX,12/05/1996 0:00,11/04/2017 0:00,...,2,5,4278,61,62,129889,4586,61,4.0,0


In [6]:
# Your comment here
websites.dtypes

URL                           object
URL_LENGTH                     int64
NUMBER_SPECIAL_CHARACTERS      int64
CHARSET                       object
SERVER                        object
CONTENT_LENGTH               float64
WHOIS_COUNTRY                 object
WHOIS_STATEPRO                object
WHOIS_REGDATE                 object
WHOIS_UPDATED_DATE            object
TCP_CONVERSATION_EXCHANGE      int64
DIST_REMOTE_TCP_PORT           int64
REMOTE_IPS                     int64
APP_BYTES                      int64
SOURCE_APP_PACKETS             int64
REMOTE_APP_PACKETS             int64
SOURCE_APP_BYTES               int64
REMOTE_APP_BYTES               int64
APP_PACKETS                    int64
DNS_QUERY_TIMES              float64
Type                           int64
dtype: object

In [7]:
websites['WHOIS_COUNTRY'].unique()

array(['None', 'US', 'SC', 'GB', 'UK', 'RU', 'AU', 'CA', 'PA', 'se', 'IN',
       'LU', 'TH', "[u'GB'; u'UK']", 'FR', 'NL', 'UG', 'JP', 'CN', 'SE',
       'SI', 'IL', 'ru', 'KY', 'AT', 'CZ', 'PH', 'BE', 'NO', 'TR', 'LV',
       'DE', 'ES', 'BR', 'us', 'KR', 'HK', 'UA', 'CH', 'United Kingdom',
       'BS', 'PK', 'IT', 'Cyprus', 'BY', 'AE', 'IE', 'UY', 'KG'],
      dtype=object)

In [8]:
websites['WHOIS_COUNTRY'] = websites['WHOIS_COUNTRY'].apply(lambda x: x.upper())

In [9]:
websites.loc[(websites['WHOIS_COUNTRY'].str.contains('UNITED KINGDOM')==True),'WHOIS_COUNTRY']='UK'
websites.loc[(websites['WHOIS_COUNTRY'].str.contains('CYPRUS')==True),'WHOIS_COUNTRY']='CY'
websites.loc[(websites['WHOIS_COUNTRY'].str.contains("[U'GB'; U'UK']")==True),'WHOIS_COUNTRY']='UK'

In [10]:
#websites.drop('WHOIS_STATEPRO', axis=1, inplace = True)

In [11]:
websites['Type'].unique()


array([1, 0])

In [12]:
websites.groupby(by='Type')['Type'].count()
# 1 should be malicious

Type
0    1565
1     216
Name: Type, dtype: int64

In [13]:
websites.isna().sum()

URL                            0
URL_LENGTH                     0
NUMBER_SPECIAL_CHARACTERS      0
CHARSET                        0
SERVER                         1
CONTENT_LENGTH               812
WHOIS_COUNTRY                  0
WHOIS_STATEPRO                 0
WHOIS_REGDATE                  0
WHOIS_UPDATED_DATE             0
TCP_CONVERSATION_EXCHANGE      0
DIST_REMOTE_TCP_PORT           0
REMOTE_IPS                     0
APP_BYTES                      0
SOURCE_APP_PACKETS             0
REMOTE_APP_PACKETS             0
SOURCE_APP_BYTES               0
REMOTE_APP_BYTES               0
APP_PACKETS                    0
DNS_QUERY_TIMES                1
Type                           0
dtype: int64

In [14]:
websites.shape

(1781, 21)

In [15]:
812/1781

0.45592363840539024

In [16]:
# Since CONTENT_LENGTH has 46% of missing values and I believe the size of the webpage 
#does not help to classify between malicious and not I will drop it


In [17]:
websites.drop('CONTENT_LENGTH',axis = 1, inplace = True)

In [18]:
# There are only two more cases of NAs so I will drop those rows
clean_websites = websites.dropna(subset=['SERVER','DNS_QUERY_TIMES'])
clean_websites.isna().sum()

URL                          0
URL_LENGTH                   0
NUMBER_SPECIAL_CHARACTERS    0
CHARSET                      0
SERVER                       0
WHOIS_COUNTRY                0
WHOIS_STATEPRO               0
WHOIS_REGDATE                0
WHOIS_UPDATED_DATE           0
TCP_CONVERSATION_EXCHANGE    0
DIST_REMOTE_TCP_PORT         0
REMOTE_IPS                   0
APP_BYTES                    0
SOURCE_APP_PACKETS           0
REMOTE_APP_PACKETS           0
SOURCE_APP_BYTES             0
REMOTE_APP_BYTES             0
APP_PACKETS                  0
DNS_QUERY_TIMES              0
Type                         0
dtype: int64

In [19]:
clean_websites.isnull().sum()

URL                          0
URL_LENGTH                   0
NUMBER_SPECIAL_CHARACTERS    0
CHARSET                      0
SERVER                       0
WHOIS_COUNTRY                0
WHOIS_STATEPRO               0
WHOIS_REGDATE                0
WHOIS_UPDATED_DATE           0
TCP_CONVERSATION_EXCHANGE    0
DIST_REMOTE_TCP_PORT         0
REMOTE_IPS                   0
APP_BYTES                    0
SOURCE_APP_PACKETS           0
REMOTE_APP_PACKETS           0
SOURCE_APP_BYTES             0
REMOTE_APP_BYTES             0
APP_PACKETS                  0
DNS_QUERY_TIMES              0
Type                         0
dtype: int64

In [20]:
clean_websites.shape

(1779, 20)

In [21]:
print(clean_websites[clean_websites['WHOIS_COUNTRY']=='None'].count()[0]/1779)
#print(clean_websites[clean_websites['WHOIS_STATEPRO']=='None'].count()[0]/1779)
print(clean_websites[clean_websites['WHOIS_REGDATE']=='None'].count()[0]/1779)
print(clean_websites[clean_websites['WHOIS_UPDATED_DATE']=='None'].count()[0]/1779)
print(clean_websites[clean_websites['URL_LENGTH']=='None'].count()[0]/1779)
print(clean_websites[clean_websites['TCP_CONVERSATION_EXCHANGE']=='None'].count()[0]/1779)
# These 4 columns have a significant percentage of 'None'. I'm inclined to drop the first two and clean the columns with date 
# But will wait for the correlations to tell me (Country and state actually can be interesting for the model)

0.0
0.0713884204609331
0.07813378302417089
0.0
0.0


/home/jonnyteronni/Documents/Programs/anaconda3/lib/python3.7/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


In [22]:
clean_websites['WHOIS_REGDATE_clean'] = pd.to_datetime(clean_websites['WHOIS_REGDATE'],errors = 'coerce')
clean_websites['WHOIS_UPDATED_DATE_clean'] = pd.to_datetime(clean_websites['WHOIS_UPDATED_DATE'],errors = 'coerce')
clean_websites.drop('WHOIS_REGDATE',inplace=True, axis = 1)
clean_websites.drop('WHOIS_UPDATED_DATE',inplace=True, axis = 1)

/home/jonnyteronni/Documents/Programs/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/home/jonnyteronni/Documents/Programs/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/jonnyteronni/Documents/Programs/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of 

In [23]:
clean_websites.head()
# Now that I think about it I don't think I need the columns with dates, but will se on the next steps.

,URL,URL_LENGTH,NUMBER_SPECIAL_CHARACTERS,CHARSET,SERVER,WHOIS_COUNTRY,WHOIS_STATEPRO,TCP_CONVERSATION_EXCHANGE,DIST_REMOTE_TCP_PORT,REMOTE_IPS,APP_BYTES,SOURCE_APP_PACKETS,REMOTE_APP_PACKETS,SOURCE_APP_BYTES,REMOTE_APP_BYTES,APP_PACKETS,DNS_QUERY_TIMES,Type,WHOIS_REGDATE_clean,WHOIS_UPDATED_DATE_clean
0,M0_109,16,7,iso-8859-1,nginx,NONE,None,7,0,2,700,9,10,1153,832,9,2.0,1,2015-10-10 18:21:00,NaT
1,B0_2314,16,6,UTF-8,Apache/2.4.10,NONE,None,17,7,4,1230,17,19,1265,1230,17,0.0,0,NaT,NaT
2,B0_911,16,6,us-ascii,Microsoft-HTTPAPI/2.0,NONE,None,0,0,0,0,0,0,0,0,0,0.0,0,NaT,NaT
3,B0_113,17,6,ISO-8859-1,nginx,UK,AK,31,22,3,3812,39,37,18784,4380,39,8.0,0,1997-07-10 04:00:00,2013-12-09 00:45:00
4,B0_403,17,6,UTF-8,None,UK,TX,57,2,5,4278,61,62,129889,4586,61,4.0,0,1996-12-05 00:00:00,2017-11-04 00:00:00


In [24]:
clean_websites.dtypes

URL                           object
URL_LENGTH                     int64
NUMBER_SPECIAL_CHARACTERS      int64
CHARSET                       object
SERVER                        object
WHOIS_COUNTRY                 object
WHOIS_STATEPRO                object
TCP_CONVERSATION_EXCHANGE      int64
DIST_REMOTE_TCP_PORT           int64
REMOTE_IPS                     int64
APP_BYTES                      int64
SOURCE_APP_PACKETS             int64
REMOTE_APP_PACKETS             int64
SOURCE_APP_BYTES               int64
REMOTE_APP_BYTES               int64
APP_PACKETS                    int64
DNS_QUERY_TIMES              float64
Type                           int64
WHOIS_REGDATE_clean           object
WHOIS_UPDATED_DATE_clean      object
dtype: object

In [25]:
clean_websites['CHARSET'].unique()

array(['iso-8859-1', 'UTF-8', 'us-ascii', 'ISO-8859-1', 'utf-8', 'None',
       'windows-1251', 'ISO-8859', 'windows-1252'], dtype=object)

In [26]:
clean_websites['SERVER'].unique()

array(['nginx', 'Apache/2.4.10', 'Microsoft-HTTPAPI/2.0', 'None',
       'Apache/2', 'nginx/1.10.1', 'Apache', 'Apache/2.2.15 (Red Hat)',
       'Apache/2.4.23 (Unix) OpenSSL/1.0.1e-fips mod_bwlimited/1.4',
       'openresty/1.11.2.1', 'Apache/2.2.22', 'Apache/2.4.7 (Ubuntu)',
       'nginx/1.12.0',
       'Apache/2.4.12 (Unix) OpenSSL/1.0.1e-fips mod_bwlimited/1.4',
       'Oracle-iPlanet-Web-Server/7.0', 'cloudflare-nginx', 'nginx/1.6.2',
       'openresty', 'Heptu web server', 'Pepyaka/1.11.3', 'nginx/1.8.0',
       'nginx/1.10.1 + Phusion Passenger 5.0.30',
       'Apache/2.2.29 (Amazon)', 'Microsoft-IIS/7.5', 'LiteSpeed',
       'Apache/2.4.25 (cPanel) OpenSSL/1.0.1e-fips mod_bwlimited/1.4',
       'tsa_c', 'Apache/2.2.0 (Fedora)', 'Apache/2.2.22 (Debian)',
       'Apache/2.2.15 (CentOS)', 'Apache/2.4.25',
       'Apache/2.4.25 (Amazon) PHP/7.0.14', 'GSE',
       'Apache/2.4.23 (Unix) OpenSSL/0.9.8e-fips-rhel5 mod_bwlimited/1.4',
       'Apache/2.4.25 (Amazon) OpenSSL/1.0.1k-fips'

In [27]:
import re

clean_websites['OS'] = clean_websites['SERVER'].str.extract(r'(\([A-z()]+\))')



/home/jonnyteronni/Documents/Programs/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [28]:
clean_websites['OS'].unique()

array([nan, '(Unix)', '(Ubuntu)', '(Amazon)', '(cPanel)', '(Fedora)',
       '(Debian)', '(CentOS)', '(FreeBSD)', '(Linux)', '(OpenBSD)'],
      dtype=object)

In [29]:
clean_websites['OS'].isna().sum()

1586

In [30]:
clean_websites.drop('OS',axis=1,inplace=True)

/home/jonnyteronni/Documents/Programs/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [31]:
clean_websites['Type']

0       1
1       0
2       0
3       0
4       0
       ..
1776    1
1777    1
1778    0
1779    0
1780    0
Name: Type, Length: 1779, dtype: int64

In [32]:
# (^([^/]+))
clean_websites['type_server'] = clean_websites['SERVER'].str.extract(r'([^/]+)')


/home/jonnyteronni/Documents/Programs/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [33]:
clean_websites[['SERVER','type_server']].head(5)

,SERVER,type_server
0,nginx,nginx
1,Apache/2.4.10,Apache
2,Microsoft-HTTPAPI/2.0,Microsoft-HTTPAPI
3,nginx,nginx
4,None,None


In [34]:
clean_websites['type_server'].unique()

array(['nginx', 'Apache', 'Microsoft-HTTPAPI', 'None', 'openresty',
       'Oracle-iPlanet-Web-Server', 'cloudflare-nginx',
       'Heptu web server', 'Pepyaka', 'Microsoft-IIS', 'LiteSpeed',
       'tsa_c', 'GSE', 'Tengine', 'Sun-ONE-Web-Server', 'AmazonS3', 'ATS',
       'CherryPy', 'Server', 'KHL', 'mw2114.codfw.wmnet',
       'Jetty(9.0.z-SNAPSHOT)', 'HTTPDaemon', 'MediaFire', 'DOSarrest',
       'mw2232.codfw.wmnet', 'Sucuri', 'mw2260.codfw.wmnet',
       'mw2239.codfw.wmnet', 'DPS', 'SSWS', 'YouTubeFrontEnd', 'Squeegit',
       'Virtuoso', 'Apache-Coyote', 'Yippee-Ki-Yay', 'mw2165.codfw.wmnet',
       'mw2192.codfw.wmnet', 'nginx + Phusion Passenger',
       'Proxy Pandeiro UOL', 'mw2231.codfw.wmnet', 'mw2109.codfw.wmnet',
       'mw2225.codfw.wmnet', 'mw2236.codfw.wmnet', 'mw2101.codfw.wmnet',
       'Varnish', 'Resin', 'mw2164.codfw.wmnet', 'mw2242.codfw.wmnet',
       'mw2175.codfw.wmnet', 'mw2107.codfw.wmnet', 'mw2190.codfw.wmnet',
       'barista', 'mw2103.codfw.wmnet', 'ECD

In [35]:
clean_websites.loc[(clean_websites['type_server'].str.contains('^mw.*')==True),'type_server']='mw'

/home/jonnyteronni/Documents/Programs/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [36]:
clean_websites[clean_websites['type_server'].str.contains('^mw.*')==True]

,URL,URL_LENGTH,NUMBER_SPECIAL_CHARACTERS,CHARSET,SERVER,WHOIS_COUNTRY,WHOIS_STATEPRO,TCP_CONVERSATION_EXCHANGE,DIST_REMOTE_TCP_PORT,REMOTE_IPS,...,SOURCE_APP_PACKETS,REMOTE_APP_PACKETS,SOURCE_APP_BYTES,REMOTE_APP_BYTES,APP_PACKETS,DNS_QUERY_TIMES,Type,WHOIS_REGDATE_clean,WHOIS_UPDATED_DATE_clean,type_server
235,B0_215,33,8,UTF-8,mw2114.codfw.wmnet,UK,CA,32,11,6,...,40,32,11543,4286,40,8.0,0,2001-01-13 00:12:00,2015-12-12 10:16:00,mw
285,B0_1148,35,8,UTF-8,mw2232.codfw.wmnet,UK,CA,0,0,0,...,0,0,0,0,0,0.0,0,2001-01-13 00:12:00,2015-12-12 10:16:00,mw
327,B0_2260,36,8,UTF-8,mw2260.codfw.wmnet,UK,CA,5,3,3,...,5,7,438,498,5,0.0,0,2001-01-13 00:12:00,2015-12-12 10:16:00,mw
331,B0_366,36,8,UTF-8,mw2239.codfw.wmnet,UK,CA,85,73,9,...,91,96,107416,7962,91,6.0,0,2001-01-13 00:12:00,2015-12-12 10:16:00,mw
511,B0_1017,40,9,UTF-8,mw2239.codfw.wmnet,UK,CA,0,0,0,...,0,0,0,0,0,0.0,0,2001-01-13 00:12:00,2015-12-12 10:16:00,mw
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1433,B0_804,73,17,UTF-8,mw2242.codfw.wmnet,UK,CA,0,0,0,...,0,0,0,0,0,0.0,0,2001-01-13 00:12:00,2015-12-12 10:16:00,mw
1444,B0_2238,75,14,UTF-8,mw2238.codfw.wmnet,UK,CA,4,3,2,...,4,6,372,432,4,0.0,0,2001-01-13 00:12:00,2015-12-12 10:16:00,mw
1464,B0_1393,78,17,UTF-8,mw2165.codfw.wmnet,UK,CA,0,0,0,...,0,0,0,0,0,0.0,0,2001-01-13 00:12:00,2015-12-12 10:16:00,mw
1470,B0_278,78,13,UTF-8,mw2224.codfw.wmnet,UK,CA,39,35,2,...,47,47,32891,6557,47,8.0,0,2001-01-13 00:12:00,2015-12-12 10:16:00,mw


In [37]:
clean_websites[clean_websites['type_server'].str.contains('^mw.*')==True]['type_server']

235     mw
285     mw
327     mw
331     mw
511     mw
        ..
1433    mw
1444    mw
1464    mw
1470    mw
1550    mw
Name: type_server, Length: 61, dtype: object

In [38]:
clean_websites['type_server'].unique()

array(['nginx', 'Apache', 'Microsoft-HTTPAPI', 'None', 'openresty',
       'Oracle-iPlanet-Web-Server', 'cloudflare-nginx',
       'Heptu web server', 'Pepyaka', 'Microsoft-IIS', 'LiteSpeed',
       'tsa_c', 'GSE', 'Tengine', 'Sun-ONE-Web-Server', 'AmazonS3', 'ATS',
       'CherryPy', 'Server', 'KHL', 'mw', 'Jetty(9.0.z-SNAPSHOT)',
       'HTTPDaemon', 'MediaFire', 'DOSarrest', 'Sucuri', 'DPS', 'SSWS',
       'YouTubeFrontEnd', 'Squeegit', 'Virtuoso', 'Apache-Coyote',
       'Yippee-Ki-Yay', 'nginx + Phusion Passenger', 'Proxy Pandeiro UOL',
       'Varnish', 'Resin', 'barista', 'ECD (fll', 'Pagely Gateway', 'fbs',
       'gunicorn', 'lighttpd', 'AkamaiGHost', 'PWS', 'nxfps', 'Play',
       'Scratch Web Server', 'marrakesh 1.12.2', 'squid',
       'Nginx (OpenBSD)', 'Zope', '.V01 Apache',
       'Aeria Games & Entertainment', 'Cowboy', 'DMS',
       'Application-Server', '294', 'www.lexisnexis.com  9999', 'Pizza',
       'XXXXXXXXXXXXXXXXXXXXXX', 'MI', 'Roxen', 'ebay server',
       'B

In [39]:
clean_websites['Type']

0       1
1       0
2       0
3       0
4       0
       ..
1776    1
1777    1
1778    0
1779    0
1780    0
Name: Type, Length: 1779, dtype: int64

In [40]:
onehotenc = pd.get_dummies(clean_websites['type_server'])
websites_data = pd.concat([clean_websites,onehotenc],axis = 1)

In [41]:
onehotenc.columns

Index(['.V01 Apache', '294', 'ATS', 'Aeria Games & Entertainment',
       'AkamaiGHost', 'AmazonS3', 'Apache', 'Apache-Coyote',
       'Application-Server', 'Boston.com Frontend', 'CherryPy', 'Cowboy',
       'DMS', 'DOSarrest', 'DPS', 'ECD (fll', 'GSE', 'HTTPDaemon',
       'Heptu web server', 'IdeaWebServer', 'Jetty(9.0.z-SNAPSHOT)', 'KHL',
       'LiteSpeed', 'MI', 'MediaFire', 'Microsoft-HTTPAPI', 'Microsoft-IIS',
       'My Arse', 'Nginx (OpenBSD)', 'None', 'Oracle-iPlanet-Web-Server',
       'PWS', 'Pagely Gateway', 'Pepyaka', 'Pizza', 'Play',
       'Proxy Pandeiro UOL', 'Resin', 'Roxen', 'SSWS', 'Scratch Web Server',
       'Server', 'Squeegit', 'Sucuri', 'Sun-ONE-Web-Server', 'Tengine',
       'Varnish', 'Virtuoso', 'XXXXXXXXXXXXXXXXXXXXXX', 'Yippee-Ki-Yay',
       'YouTubeFrontEnd', 'Zope', 'barista', 'cloudflare-nginx', 'ebay server',
       'fbs', 'gunicorn', 'lighttpd', 'marrakesh 1.12.2', 'mw', 'nginx',
       'nginx + Phusion Passenger', 'nxfps', 'openresty', 'squid', 't

In [42]:
websites_data.drop('SERVER',axis=1,inplace=True)
websites_data.drop('type_server',axis=1,inplace=True)

In [43]:
onehotenc = pd.get_dummies(websites_data['CHARSET'])
websites_data = pd.concat([websites_data,onehotenc],axis = 1)
websites_data.drop('CHARSET',axis=1,inplace=True)

In [44]:
onehotenc = pd.get_dummies(websites_data['WHOIS_COUNTRY'])
websites_data = pd.concat([websites_data,onehotenc],axis = 1)
websites_data.drop('WHOIS_COUNTRY',axis=1,inplace=True)

In [45]:
onehotenc = pd.get_dummies(websites_data['WHOIS_STATEPRO'])
websites_data = pd.concat([websites_data,onehotenc],axis = 1)
websites_data.drop('WHOIS_STATEPRO',axis=1,inplace=True)

In [46]:
websites_data.head()

,URL,URL_LENGTH,NUMBER_SPECIAL_CHARACTERS,TCP_CONVERSATION_EXCHANGE,DIST_REMOTE_TCP_PORT,REMOTE_IPS,APP_BYTES,SOURCE_APP_PACKETS,REMOTE_APP_PACKETS,SOURCE_APP_BYTES,...,kireka,liaoningsheng,nj,ny,qc,quebec,shandong,va,widestep@mail.ru,worcs
0,M0_109,16,7,7,0,2,700,9,10,1153,...,0,0,0,0,0,0,0,0,0,0
1,B0_2314,16,6,17,7,4,1230,17,19,1265,...,0,0,0,0,0,0,0,0,0,0
2,B0_911,16,6,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,B0_113,17,6,31,22,3,3812,39,37,18784,...,0,0,0,0,0,0,0,0,0,0
4,B0_403,17,6,57,2,5,4278,61,62,129889,...,0,0,0,0,0,0,0,0,0,0


In [47]:
target = websites_data['Type']
features = websites_data.drop('Type',axis=1)

#### Next, evaluate if the columns in this dataset are strongly correlated.

In the Mushroom supervised learning lab we did recently, we mentioned we are concerned if our dataset has strongly correlated columns because if it is the case we need to choose certain ML algorithms instead of others. We need to evaluate this for our dataset now.

Luckily, most of the columns in this dataset are ordinal which makes things a lot easier for us. In the next cells below, evaluate the level of collinearity of the data.

We provide some general directions for you to consult in order to complete this step:

1. You will create a correlation matrix using the numeric columns in the dataset.

1. Create a heatmap using `seaborn` to visualize which columns have high collinearity.

1. Comment on which columns you might need to remove due to high collinearity.

In [48]:
# Your code here
for i in websites_data.columns:
    print(i)


URL
URL_LENGTH
NUMBER_SPECIAL_CHARACTERS
TCP_CONVERSATION_EXCHANGE
DIST_REMOTE_TCP_PORT
REMOTE_IPS
APP_BYTES
SOURCE_APP_PACKETS
REMOTE_APP_PACKETS
SOURCE_APP_BYTES
REMOTE_APP_BYTES
APP_PACKETS
DNS_QUERY_TIMES
Type
WHOIS_REGDATE_clean
WHOIS_UPDATED_DATE_clean
.V01 Apache
294
ATS
Aeria Games & Entertainment
AkamaiGHost
AmazonS3
Apache
Apache-Coyote
Application-Server
Boston.com Frontend
CherryPy
Cowboy
DMS
DOSarrest
DPS
ECD (fll
GSE
HTTPDaemon
Heptu web server
IdeaWebServer
Jetty(9.0.z-SNAPSHOT)
KHL
LiteSpeed
MI
MediaFire
Microsoft-HTTPAPI
Microsoft-IIS
My Arse
Nginx (OpenBSD)
None
Oracle-iPlanet-Web-Server
PWS
Pagely Gateway
Pepyaka
Pizza
Play
Proxy Pandeiro UOL
Resin
Roxen
SSWS
Scratch Web Server
Server
Squeegit
Sucuri
Sun-ONE-Web-Server
Tengine
Varnish
Virtuoso
XXXXXXXXXXXXXXXXXXXXXX
Yippee-Ki-Yay
YouTubeFrontEnd
Zope
barista
cloudflare-nginx
ebay server
fbs
gunicorn
lighttpd
marrakesh 1.12.2
mw
nginx
nginx + Phusion Passenger
nxfps
openresty
squid
tsa_c
www.lexisnexis.com  9999
ISO-8

In [49]:
websites_data['Type'].head()

0    1
1    0
2    0
3    0
4    0
Name: Type, dtype: int64

In [50]:
websites_data.corr()

,URL_LENGTH,NUMBER_SPECIAL_CHARACTERS,TCP_CONVERSATION_EXCHANGE,DIST_REMOTE_TCP_PORT,REMOTE_IPS,APP_BYTES,SOURCE_APP_PACKETS,REMOTE_APP_PACKETS,SOURCE_APP_BYTES,REMOTE_APP_BYTES,...,kireka,liaoningsheng,nj,ny,qc,quebec,shandong,va,widestep@mail.ru,worcs
URL_LENGTH,1.000000,0.918098,-0.038159,-0.039696,-0.046174,-0.026416,-0.041967,-0.033481,-0.014628,-0.026652,...,-0.024050,-0.017162,0.006582,0.002642,-0.000802,-0.028356,-0.018023,0.068189,-0.006829,-0.013718
NUMBER_SPECIAL_CHARACTERS,0.918098,1.000000,-0.037323,-0.042554,-0.046961,-0.023896,-0.039939,-0.030467,-0.014276,-0.024079,...,-0.021419,-0.005783,0.001465,-0.000571,-0.010995,-0.026631,-0.016207,0.070388,0.004641,-0.016207
TCP_CONVERSATION_EXCHANGE,-0.038159,-0.037323,1.000000,0.555180,0.331057,0.457324,0.997798,0.990871,0.865585,0.458705,...,-0.006597,-0.002499,0.001443,0.049602,-0.009524,-0.006011,0.001014,-0.003764,-0.002499,-0.009524
DIST_REMOTE_TCP_PORT,-0.039696,-0.042554,0.555180,1.000000,0.210198,0.780243,0.558601,0.591173,0.313333,0.781216,...,-0.005954,-0.005954,0.005491,-0.001605,-0.005954,-0.001605,-0.003780,-0.002239,0.002743,-0.005954
REMOTE_IPS,-0.046174,-0.046961,0.331057,0.210198,1.000000,0.023124,0.361087,0.304712,0.171660,0.025322,...,-0.007425,-0.007425,0.027174,0.083580,-0.021426,0.006575,0.034577,-0.003684,-0.014426,-0.021426
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
quebec,-0.028356,-0.026631,-0.006011,-0.001605,0.006575,-0.001024,-0.007148,-0.005500,-0.005227,-0.001097,...,-0.000562,-0.000562,-0.001126,-0.000562,-0.000562,1.000000,-0.000562,-0.001594,-0.000562,-0.000562
shandong,-0.018023,-0.016207,0.001014,-0.003780,0.034577,-0.000657,0.000826,0.004210,-0.004682,-0.000671,...,-0.000562,-0.000562,-0.001126,-0.000562,-0.000562,-0.000562,1.000000,-0.001594,-0.000562,-0.000562
va,0.068189,0.070388,-0.003764,-0.002239,-0.003684,-0.001681,-0.001292,-0.003819,-0.008121,-0.001535,...,-0.001594,-0.001594,-0.003191,-0.001594,-0.001594,-0.001594,-0.001594,1.000000,-0.001594,-0.001594
widestep@mail.ru,-0.006829,0.004641,-0.002499,0.002743,-0.014426,-0.000401,-0.001453,-0.001412,-0.003300,-0.000340,...,-0.000562,-0.000562,-0.001126,-0.000562,-0.000562,-0.000562,-0.000562,-0.001594,1.000000,-0.000562


In [51]:
# list of columns to remove based on threshold
to_remove = [column for column in abs(websites_data.corr()['Type'])<0.1]
websites_data.corr()[to_remove].index
df = websites_data.drop(websites_data.corr()[to_remove].index, axis = 1)

df.shape

(1779, 17)

In [52]:
to_remove

[False,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 False,
 True,
 True,
 True,

In [53]:
df.head()

,URL,URL_LENGTH,NUMBER_SPECIAL_CHARACTERS,Type,WHOIS_REGDATE_clean,WHOIS_UPDATED_DATE_clean,Apache,nginx,UTF-8,utf-8,CZ,ES,Barcelona,Krasnoyarsk,PRAHA,Utah,WC1N
0,M0_109,16,7,1,2015-10-10 18:21:00,NaT,0,1,0,0,0,0,0,0,0,0,0
1,B0_2314,16,6,0,NaT,NaT,1,0,1,0,0,0,0,0,0,0,0
2,B0_911,16,6,0,NaT,NaT,0,0,0,0,0,0,0,0,0,0,0
3,B0_113,17,6,0,1997-07-10 04:00:00,2013-12-09 00:45:00,0,1,0,0,0,0,0,0,0,0,0
4,B0_403,17,6,0,1996-12-05 00:00:00,2017-11-04 00:00:00,0,0,1,0,0,0,0,0,0,0,0


In [54]:
df.corr()

,URL_LENGTH,NUMBER_SPECIAL_CHARACTERS,Type,Apache,nginx,UTF-8,utf-8,CZ,ES,Barcelona,Krasnoyarsk,PRAHA,Utah,WC1N
URL_LENGTH,1.000000,0.918098,0.162641,0.103380,-0.096608,0.164465,-0.021782,-0.018521,0.460726,0.466175,-0.020995,-0.016914,-0.046273,-0.052881
NUMBER_SPECIAL_CHARACTERS,0.918098,1.000000,0.281150,0.098462,-0.049161,0.201719,-0.020680,0.019177,0.475296,0.481069,-0.022009,0.024230,-0.025274,-0.042874
Type,0.162641,0.281150,1.000000,0.168251,0.110165,0.177976,-0.109356,0.167559,0.506112,0.511168,0.127698,0.180796,0.230796,0.127698
Apache,0.103380,0.098462,0.168251,1.000000,-0.354017,0.045519,-0.121820,0.097374,0.255269,0.259489,-0.034763,0.091779,0.117161,0.064825
nginx,-0.096608,-0.049161,0.110165,-0.354017,1.000000,0.051225,0.063781,-0.034472,-0.092628,-0.091864,-0.022949,-0.032491,-0.041477,-0.022949
UTF-8,0.164465,0.201719,0.177976,0.045519,0.051225,1.000000,-0.406354,0.074970,0.239067,0.243311,-0.037075,0.086057,-0.067008,-0.012610
utf-8,-0.021782,-0.020680,-0.109356,-0.121820,0.063781,-0.406354,1.000000,-0.037101,-0.099694,-0.098870,-0.024699,-0.034970,-0.044641,-0.024699
CZ,-0.018521,0.019177,0.167559,0.097374,-0.034472,0.074970,-0.037101,1.000000,-0.013663,-0.013550,-0.003385,0.942543,-0.006118,-0.003385
ES,0.460726,0.475296,0.506112,0.255269,-0.092628,0.239067,-0.099694,-0.013663,1.000000,0.991743,-0.009096,-0.012878,-0.016439,-0.009096
Barcelona,0.466175,0.481069,0.511168,0.259489,-0.091864,0.243311,-0.098870,-0.013550,0.991743,1.000000,-0.009021,-0.012772,-0.016304,-0.009021


In [55]:
target = df['Type']
features = df.drop('Type',axis=1)

# Challenge 2 - Remove Column Collinearity.

From the heatmap you created, you should have seen at least 3 columns that can be removed due to high collinearity. Remove these columns from the dataset.

Note that you should remove as few columns as you can. You don't have to remove all the columns at once. But instead, try removing one column, then produce the heatmap again to determine if additional columns should be removed. As long as the dataset no longer contains columns that are correlated for over 90%, you can stop. Also, keep in mind when two columns have high collinearity, you only need to remove one of them but not both.

In the cells below, remove as few columns as you can to eliminate the high collinearity in the dataset. Make sure to comment on your way so that the instructional team can learn about your thinking process which allows them to give feedback. At the end, print the heatmap again.

In [56]:
# Your code here


In [57]:
# Your comment here

In [58]:
# Print heatmap again


# Challenge 3 - Handle Missing Values

The next step would be handling missing values. **We start by examining the number of missing values in each column, which you will do in the next cell.**

In [59]:
# Your code here


If you remember in the previous labs, we drop a column if the column contains a high proportion of missing values. After dropping those problematic columns, we drop the rows with missing values.

#### In the cells below, handle the missing values from the dataset. Remember to comment the rationale of your decisions.

In [60]:
# Your code here


In [61]:
# Your comment here

#### Again, examine the number of missing values in each column. 

If all cleaned, proceed. Otherwise, go back and do more cleaning.

In [62]:
# Examine missing values in each column


# Challenge 4 - Handle `WHOIS_*` Categorical Data

There are several categorical columns we need to handle. These columns are:

* `URL`
* `CHARSET`
* `SERVER`
* `WHOIS_COUNTRY`
* `WHOIS_STATEPRO`
* `WHOIS_REGDATE`
* `WHOIS_UPDATED_DATE`

How to handle string columns is always case by case. Let's start by working on `WHOIS_COUNTRY`. Your steps are:

1. List out the unique values of `WHOIS_COUNTRY`.
1. Consolidate the country values with consistent country codes. For example, the following values refer to the same country and should use consistent country code:
    * `CY` and `Cyprus`
    * `US` and `us`
    * `SE` and `se`
    * `GB`, `United Kingdom`, and `[u'GB'; u'UK']`

#### In the cells below, fix the country values as intructed above.

In [63]:
# Your code here


Since we have fixed the country values, can we convert this column to ordinal now?

Not yet. If you reflect on the previous labs how we handle categorical columns, you probably remember we ended up dropping a lot of those columns because there are too many unique values. Too many unique values in a column is not desirable in machine learning because it makes prediction inaccurate. But there are workarounds under certain conditions. One of the fixable conditions is:

#### If a limited number of values account for the majority of data, we can retain these top values and re-label all other rare values.

The `WHOIS_COUNTRY` column happens to be this case. You can verify it by print a bar chart of the `value_counts` in the next cell to verify:

In [64]:
# Your code here


#### After verifying, now let's keep the top 10 values of the column and re-label other columns with `OTHER`.

In [65]:
# Your code here


Now since `WHOIS_COUNTRY` has been re-labelled, we don't need `WHOIS_STATEPRO` any more because the values of the states or provinces may not be relevant any more. We'll drop this column.

In addition, we will also drop `WHOIS_REGDATE` and `WHOIS_UPDATED_DATE`. These are the registration and update dates of the website domains. Not of our concerns.

#### In the next cell, drop `['WHOIS_STATEPRO', 'WHOIS_REGDATE', 'WHOIS_UPDATED_DATE']`.

In [66]:
# Your code here


# Challenge 5 - Handle Remaining Categorical Data & Convert to Ordinal

Now print the `dtypes` of the data again. Besides `WHOIS_COUNTRY` which we already fixed, there should be 3 categorical columns left: `URL`, `CHARSET`, and `SERVER`.

In [67]:
# Your code here


#### `URL` is easy. We'll simply drop it because it has too many unique values that there's no way for us to consolidate.

In [68]:
# Your code here
features.drop('URL',axis=1,inplace=True)
features.drop('WHOIS_REGDATE_clean',axis=1,inplace=True)
features.drop('WHOIS_UPDATED_DATE_clean',axis=1,inplace=True)

In [69]:
features.dtypes

URL_LENGTH                   int64
NUMBER_SPECIAL_CHARACTERS    int64
Apache                       uint8
nginx                        uint8
UTF-8                        uint8
utf-8                        uint8
CZ                           uint8
ES                           uint8
Barcelona                    uint8
Krasnoyarsk                  uint8
PRAHA                        uint8
Utah                         uint8
WC1N                         uint8
dtype: object

In [70]:
for i in features.columns:
    print(i)

URL_LENGTH
NUMBER_SPECIAL_CHARACTERS
Apache
nginx
UTF-8
utf-8
CZ
ES
Barcelona
Krasnoyarsk
PRAHA
Utah
WC1N


In [71]:
for i in features[features=='None'].count():
    print(i)


0
0
0
0
0
0
0
0
0
0
0
0
0


/home/jonnyteronni/Documents/Programs/anaconda3/lib/python3.7/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


#### Print the unique value counts of `CHARSET`. You see there are only a few unique values. So we can keep it as it is.

In [72]:
# Your code here

`SERVER` is a little more complicated. Print its unique values and think about how you can consolidate those values.

#### Before you think of your own solution, don't read the instructions that come next.

In [73]:
# Your code here


![Think Hard](../images/think-hard.jpg)

In [74]:
# Your comment here


Although there are so many unique values in the `SERVER` column, there are actually only 3 main server types: `Microsoft`, `Apache`, and `nginx`. Just check if each `SERVER` value contains any of those server types and re-label them. For `SERVER` values that don't contain any of those substrings, label with `Other`.

At the end, your `SERVER` column should only contain 4 unique values: `Microsoft`, `Apache`, `nginx`, and `Other`.

In [75]:
# Your code here


In [76]:
# Count `SERVER` value counts here


OK, all our categorical data are fixed now. **Let's convert them to ordinal data using Pandas' `get_dummies` function ([documentation](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html)). Also, assign the data with dummy values to a new variable `website_dummy`.**

In [77]:
# Your code here


Now, inspect `website_dummy` to make sure the data and types are intended - there shouldn't be any categorical columns at this point.

In [78]:
# Your code here


# Challenge 6 - Modeling, Prediction, and Evaluation

We'll start off this section by splitting the data to train and test. **Name your 4 variables `X_train`, `X_test`, `y_train`, and `y_test`. Select 80% of the data for training and 20% for testing.**

In [79]:
from sklearn.model_selection import train_test_split

# Your code here:

X_train, X_test, y_train, y_test = train_test_split(features, target,test_size =0.2, random_state=0)


#### In this lab, we will try two different models and compare our results.

The first model we will use in this lab is logistic regression. We have previously learned about logistic regression as a classification algorithm. In the cell below, load `LogisticRegression` from scikit-learn and initialize the model.

In [80]:
# Your code here:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(max_iter=2000)



Next, fit the model to our training data. We have already separated our data into 4 parts. Use those in your model.

In [81]:
# Your code here:
model.fit(X_train, y_train)



LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=2000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [82]:
for i in features.columns:
    print(i)

URL_LENGTH
NUMBER_SPECIAL_CHARACTERS
Apache
nginx
UTF-8
utf-8
CZ
ES
Barcelona
Krasnoyarsk
PRAHA
Utah
WC1N


finally, import `confusion_matrix` and `accuracy_score` from `sklearn.metrics` and fit our testing data. Assign the fitted data to `y_pred` and print the confusion matrix as well as the accuracy score

In [83]:
# Your code here:
from sklearn.metrics import confusion_matrix

from sklearn.metrics import classification_report 

confusion_matrix(model.predict(X_test),y_test)


array([[316,  21],
       [  1,  18]])

In [84]:
print(classification_report(y_test, model.predict(X_test)))

              precision    recall  f1-score   support

           0       0.94      1.00      0.97       317
           1       0.95      0.46      0.62        39

    accuracy                           0.94       356
   macro avg       0.94      0.73      0.79       356
weighted avg       0.94      0.94      0.93       356



What are your thoughts on the performance of the model? Write your conclusions below.

In [85]:
# Your conclusions here:
print(model.score(X_train, y_train))
print(model.score(X_test, y_test))


0.9262122276879832
0.9382022471910112


#### Our second algorithm is is K-Nearest Neighbors. 

Though is it not required, we will fit a model using the training data and then test the performance of the model using the testing data. Start by loading `KNeighborsClassifier` from scikit-learn and then initializing and fitting the model. We'll start off with a model where k=3.

In [1444]:
# Your code here:



To test your model, compute the predicted values for the testing sample and print the confusion matrix as well as the accuracy score.

In [1445]:
# Your code here:



#### We'll create another K-Nearest Neighbors model with k=5. 

Initialize and fit the model below and print the confusion matrix and the accuracy score.

In [1446]:
# Your code here:



Did you see an improvement in the confusion matrix when increasing k to 5? Did you see an improvement in the accuracy score? Write your conclusions below.

In [1447]:
# Your conclusions here:



# Bonus Challenge - Feature Scaling

Problem-solving in machine learning is iterative. You can improve your model prediction with various techniques (there is a sweetspot for the time you spend and the improvement you receive though). Now you've completed only one iteration of ML analysis. There are more iterations you can conduct to make improvements. In order to be able to do that, you will need deeper knowledge in statistics and master more data analysis techniques. In this bootcamp, we don't have time to achieve that advanced goal. But you will make constant efforts after the bootcamp to eventually get there.

However, now we do want you to learn one of the advanced techniques which is called *feature scaling*. The idea of feature scaling is to standardize/normalize the range of independent variables or features of the data. This can make the outliers more apparent so that you can remove them. This step needs to happen during Challenge 6 after you split the training and test data because you don't want to split the data again which makes it impossible to compare your results with and without feature scaling. For general concepts about feature scaling, click [here](https://en.wikipedia.org/wiki/Feature_scaling). To read deeper, click [here](https://medium.com/greyatom/why-how-and-when-to-scale-your-features-4b30ab09db5e).

In the next cell, attempt to improve your model prediction accuracy by means of feature scaling. A library you can utilize is `sklearn.preprocessing.RobustScaler` ([documentation](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.RobustScaler.html)). You'll use the `RobustScaler` to fit and transform your `X_train`, then transform `X_test`. You will use logistic regression to fit and predict your transformed data and obtain the accuracy score in the same way. Compare the accuracy score with your normalized data with the previous accuracy data. Is there an improvement?

In [1448]:
# Your code here